## search history 에 대해 pareto frontier 시각화 - GA, Random search 각각

In [ ]:
# Modularize
import json
import os
import numpy as np
import plotly
import plotly.graph_objects as go


def json2graph(path, gr_type):
    # path : log file path
    # gr_type 0 : acc , 1: flops
    
    if gr_type == 0 : 
        gr_name ="GA acc"
    else : gr_name = "GA flops"
    
    with open(os.path.join(data_path,"train_logging.log")) as json_file:
        data = json.load(json_file)
        
    train_log = data['train_log']
    niter = len(train_log)
    npop = len(train_log['0'])
    objs =[]

    for i in range(niter):
        temp = [train_log[str(i)][j][1] for j in range(npop)]
        objs.append(temp)

    objs = np.array(objs)
    epoch = list(range(niter))

    # 0 : acc , 1: Flops
    idx = gr_type
    if gr_type == 0 : objs = -1*objs
        
    y1_lower = np.min(objs[:,:,idx],axis=-1)
    y1_lower = list(y1_lower[::-1])

    y1 = np.mean(objs[:,:,idx],axis=-1)

    y1_upper = np.max(objs[:,:,idx],axis=-1)
    y1_upper = list(y1_upper[::-1])


    x = [i+1 for i in range(niter)]
    x_rev = x[::-1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x+x_rev,
        y=y1_upper+ y1_lower,
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name=gr_name,
    ))

    fig.add_trace(go.Scatter(
        x=x, y=y1,
        line_color='rgb(0,100,80)',
        name=gr_name,
    ))

    fig.update_traces(mode='lines')
    fig.show()

## 1. GA - pareto frontier

In [109]:
import numpy as np
import os
import json
import sys
import pandas as pd
sys.path.insert(0,'/root/data/basic_model/')

import matplotlib.pyplot as plt
%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

from utils_kyy.create_toolbox import evaluate_v2

In [7]:
os.getcwd()

'/root/data/basic_model/notebooks'

In [72]:
data_path = '../logs/__New_main_experiment_1_GA_30gen/'

with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]
gen_num = []
for i in range(niter):
    gen_num.extend([i for j in range(npop)])
    temp = [train_log[str(i)][j][1] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))

# 0 : acc , 1: Flops
gr_type = 0

objs[:,:,0]= -1*objs[:,:,0]

y1 = objs[:,:,0].reshape(-1).tolist()  # val_accuracy 는 - 붙어있는채로 사용 => minimize 하는 pareto frontier 찾는 함수 그대로 사용
y2 = objs[:,:,1].reshape(-1).tolist()
idxs = [i for i in range(len(y1))]
pareto = [0 for i in range(len(y1))]

In [73]:
## Data Frame으로 만들기
# 1. 앞에 10 Generation 에서 파레토 만들기
# 2. 0 ~ 20 파레토 만들기
# 3. 0 ~ 30 파레토 만들기

df = pd.DataFrame({'gen':gen_num,'name': idxs, 'acc':y1, 'flops': y2})

In [74]:
df.head()

,acc,flops,gen,name
0,83.75,1.654448e+09,0,0
1,81.71,1.775048e+09,0,1
2,82.22,1.480460e+09,0,2
3,78.12,1.739114e+09,0,3
4,82.56,2.024451e+09,0,4


In [75]:
## ~ 10 gen, ~ 20 gen, ~ 30 gen 데이터 나눈 후에 각각 파레토 프론티어 구하기
idx1 = 20*11  # 0 ~ 10 gen => 총 11 gen  (0 gen 은 initialization phase)
idx2 = 20*21  # 0 ~ 20 gen
idx3 = 20*31  # = 620

# df 에서 0~idx 행까지 뽑은 뒤 ['acc', 'flops'] 컬럼만 가져와서 np array로 변환
data_10gen = df[:idx1][['acc','flops']].values
data_20gen = df[:idx2][['acc','flops']].values
data_30gen = df[:idx3][['acc','flops']].values  # df 전부

print(data_10gen.shape, data_20gen.shape, data_30gen.shape)

(220, 2) (420, 2) (620, 2)


~ 10gen, ~20gen, ~30gen 각각 파레토 프론티어 구해서 시각화 진행

In [78]:
# [Reference - 사용법] https://pythonhealthcare.org/tag/pareto-front/
# Scores (np array)를 input으로 받아서, 해당 scores 에서의 파레토 프론티어에 해당하는 점들의 인덱스를 반환해주는 함수
# e.g. scores = (300, 2) np array
#       => 점이 300개 & 최대화해야하는 objective 가 2개
#    output = [1, 3, 5, 150, 199]
#       => scores[1], scores[3], ..., scores[199] 가 파레토 프론티어임
def identify_pareto(scores):
    # Count number of items
    population_size = scores.shape[0]
    # Create a NumPy index for scores on the pareto front (zero indexed)
    population_ids = np.arange(population_size)
    # Create a starting list of items on the Pareto front
    # All items start off as being labelled as on the Parteo front
    pareto_front = np.ones(population_size, dtype=bool)
    # Loop through each item. This will then be compared with all other items
    for i in range(population_size):
        # Loop through all other items
        for j in range(population_size):
            # Check if our 'i' pint is dominated by out 'j' point
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                # j dominates i. Label 'i' point as not on Pareto front
                pareto_front[i] = 0
                # Stop further comparisons with 'i' (no more comparisons needed)
                break
    # Return ids of scenarios on pareto front
    return population_ids[pareto_front]

In [102]:
####################################################################################
# flops 에 - 를 붙여서, (val_acc, -flops) 둘 다 최대화하는 pareto frontier 탐색
####################################################################################
# 1. flops 에 - 붙이기
data_10gen_score = np.copy(data_10gen)
data_20gen_score = np.copy(data_20gen)
data_30gen_score = np.copy(data_30gen)

data_10gen_score[:, 1] = -data_10gen_score[:, 1]
data_20gen_score[:, 1] = -data_20gen_score[:, 1]
data_30gen_score[:, 1] = -data_30gen_score[:, 1]

# 2. Pareto frontier 찾기
pareto_10gen_idx = identify_pareto(data_10gen_score)  # e.g. = [ 10  27 136 142 164 167 181] => 이런 식으로 프론티어에 해당하는 인덱스를 반환
pareto_front_10gen = data_10gen_score[pareto_10gen_idx]

pareto_20gen_idx = identify_pareto(data_20gen_score)
pareto_front_20gen = data_20gen_score[pareto_20gen_idx]

pareto_30gen_idx = identify_pareto(data_30gen_score)
pareto_front_30gen = data_30gen_score[pareto_30gen_idx]

# 3. Pareto frontier 를 선분으로 긋기 위해, 각각 sorting
pareto_front_10gen_df = pd.DataFrame(pareto_front_10gen)
pareto_front_10gen_df.sort_values(0, inplace=True)
pareto_front_10gen_sorted = pareto_front_10gen_df.values

pareto_front_20gen_df = pd.DataFrame(pareto_front_20gen)
pareto_front_20gen_df.sort_values(0, inplace=True)
pareto_front_20gen_sorted = pareto_front_20gen_df.values

pareto_front_30gen_df = pd.DataFrame(pareto_front_30gen)
pareto_front_30gen_df.sort_values(0, inplace=True)
pareto_front_30gen_sorted = pareto_front_30gen_df.values

In [114]:
# 4. Plot
fig = go.Figure()

# Points: ~ 30 gen
fig.add_trace(go.Scatter(x=-data_30gen_score[:, 1], y=data_30gen_score[:, 0],
                    mode='markers',
                    name='markers'))

# Line; Pareto Frontier ~ 10 gen
# -[:, 1] = -flops => flops,  [:, 0] = val_accuracy
fig.add_trace(go.Scatter(x=-pareto_front_10gen_sorted[:, 1], y=pareto_front_10gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 10 gen'))

# Line; Pareto Frontier ~ 20 gen
fig.add_trace(go.Scatter(x=-pareto_front_20gen_sorted[:, 1], y=pareto_front_20gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 20 gen'))

# Line; Pareto Frontier ~ 30 gen
fig.add_trace(go.Scatter(x=-pareto_front_30gen_sorted[:, 1], y=pareto_front_30gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 30 gen'))

# Figure Layout
fig.update_layout(
    title='GA history & Pareto Frontier',
    xaxis=dict(
        title='Flops',
    ),
    yaxis=dict(
        title='Validation Accuracy',
    ),
)

fig.show()

##### 함수화 진행 

In [115]:
# [주의] 최소 30 generation 이상 진행되어 있어야함
def make_score_array(data_path): 
    with open(os.path.join(data_path,"train_logging.log")) as json_file:
        data = json.load(json_file)

    train_log = data['train_log']
    niter = len(train_log)
    npop = len(train_log['0'])
    objs =[]
    gen_num = []
    for i in range(niter):
        gen_num.extend([i for j in range(npop)])
        temp = [train_log[str(i)][j][1] for j in range(npop)]
        objs.append(temp)

    objs = np.array(objs)
    epoch = list(range(niter))

    # 0 : acc , 1: Flops
    gr_type = 0

    objs[:,:,0]= -1*objs[:,:,0]

    y1 = objs[:,:,0].reshape(-1).tolist()  # val_accuracy 는 - 붙어있는채로 사용 => minimize 하는 pareto frontier 찾는 함수 그대로 사용
    y2 = objs[:,:,1].reshape(-1).tolist()
    idxs = [i for i in range(len(y1))]
    pareto = [0 for i in range(len(y1))]
    
    df = pd.DataFrame({'gen':gen_num,'name': idxs, 'acc':y1, 'flops': y2})
    
    ## ~ 10 gen, ~ 20 gen, ~ 30 gen 데이터 나눈 후에 각각 파레토 프론티어 구하기
    idx1 = 20*11  # 0 ~ 10 gen => 총 11 gen  (0 gen 은 initialization phase)
    idx2 = 20*21  # 0 ~ 20 gen
    idx3 = 20*31  # = 620

    # df 에서 0~idx 행까지 뽑은 뒤 ['acc', 'flops'] 컬럼만 가져와서 np array로 변환
    data_10gen = df[:idx1][['acc','flops']].values
    data_20gen = df[:idx2][['acc','flops']].values
    data_30gen = df[:idx3][['acc','flops']].values  # df 전부

    return data_10gen, data_20gen, data_30gen

In [119]:
def find_pareto_frontier_from_scores(data_10gen_score, data_20gen_score, data_30gen_score):
    # 1. Pareto frontier 찾기
    pareto_10gen_idx = identify_pareto(data_10gen_score)  # e.g. = [ 10  27 136 142 164 167 181] => 이런 식으로 프론티어에 해당하는 인덱스를 반환
    pareto_front_10gen = data_10gen_score[pareto_10gen_idx]

    pareto_20gen_idx = identify_pareto(data_20gen_score)
    pareto_front_20gen = data_20gen_score[pareto_20gen_idx]

    pareto_30gen_idx = identify_pareto(data_30gen_score)
    pareto_front_30gen = data_30gen_score[pareto_30gen_idx]

    # 2. Pareto frontier 를 선분으로 긋기 위해, 각각 sorting
    pareto_front_10gen_df = pd.DataFrame(pareto_front_10gen)
    pareto_front_10gen_df.sort_values(0, inplace=True)
    pareto_front_10gen_sorted = pareto_front_10gen_df.values

    pareto_front_20gen_df = pd.DataFrame(pareto_front_20gen)
    pareto_front_20gen_df.sort_values(0, inplace=True)
    pareto_front_20gen_sorted = pareto_front_20gen_df.values

    pareto_front_30gen_df = pd.DataFrame(pareto_front_30gen)
    pareto_front_30gen_df.sort_values(0, inplace=True)
    pareto_front_30gen_sorted = pareto_front_30gen_df.values
    
    return pareto_front_10gen_sorted, pareto_front_20gen_sorted, pareto_front_30gen_sorted

##### [1] GA search history & Pareto Frontier

In [123]:
# 1. data_path => 각 gen 마다의 df 구축
data_path = '../logs/__New_main_experiment_1_GA_30gen/'
data_10gen, data_20gen, data_30gen = make_score_array(data_path)

# 2. flops 에 - 붙여서, 'score'로 변환 (=> 둘 다 최대화하도록)
data_10gen_score = np.copy(data_10gen)
data_20gen_score = np.copy(data_20gen)
data_30gen_score = np.copy(data_30gen)

data_10gen_score[:, 1] = -data_10gen_score[:, 1]
data_20gen_score[:, 1] = -data_20gen_score[:, 1]
data_30gen_score[:, 1] = -data_30gen_score[:, 1]

# 3. score df 로 변환 후 pareto frontier 구하기
pareto_front_sorted = find_pareto_frontier_from_scores(data_10gen_score, data_20gen_score, data_30gen_score)
pareto_front_10gen_sorted, pareto_front_20gen_sorted, pareto_front_30gen_sorted = pareto_front_sorted

# 4. Plot
title = 'GA history & Pareto Frontier'
x_title = 'Flops'
y_title = 'Validation Accuracy'

fig = go.Figure()

# Points: ~ 30 gen
fig.add_trace(go.Scatter(x=-data_30gen_score[:, 1], y=data_30gen_score[:, 0],
                    mode='markers',
                    name='markers'))

# Line; Pareto Frontier ~ 10 gen
# -[:, 1] = -flops => flops,  [:, 0] = val_accuracy
fig.add_trace(go.Scatter(x=-pareto_front_10gen_sorted[:, 1], y=pareto_front_10gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 10 gen'))

# Line; Pareto Frontier ~ 20 gen
fig.add_trace(go.Scatter(x=-pareto_front_20gen_sorted[:, 1], y=pareto_front_20gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 20 gen'))

# Line; Pareto Frontier ~ 30 gen
fig.add_trace(go.Scatter(x=-pareto_front_30gen_sorted[:, 1], y=pareto_front_30gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 30 gen'))

# Figure Layout
fig.update_layout(
    title=title,
    xaxis=dict(
        title=x_title,
    ),
    yaxis=dict(
        title=y_title,
    ),
)

fig.show()

##### [2] Random search history & Pareto Frontier

In [125]:
# 1. data_path => 각 gen 마다의 df 구축
data_path = '../logs/__New_main_experiment_1_RandomSerach_30gen/'
data_10gen, data_20gen, data_30gen = make_score_array(data_path)

# 2. flops 에 - 붙여서, 'score'로 변환 (=> 둘 다 최대화하도록)
data_10gen_score = np.copy(data_10gen)
data_20gen_score = np.copy(data_20gen)
data_30gen_score = np.copy(data_30gen)

data_10gen_score[:, 1] = -data_10gen_score[:, 1]
data_20gen_score[:, 1] = -data_20gen_score[:, 1]
data_30gen_score[:, 1] = -data_30gen_score[:, 1]

# 3. score df 로 변환 후 pareto frontier 구하기
pareto_front_sorted = find_pareto_frontier_from_scores(data_10gen_score, data_20gen_score, data_30gen_score)
pareto_front_10gen_sorted, pareto_front_20gen_sorted, pareto_front_30gen_sorted = pareto_front_sorted

# 4. Plot
title = 'Random Search history & Pareto Frontier'
x_title = 'Flops'
y_title = 'Validation Accuracy'

fig = go.Figure()

# Points: ~ 30 gen
fig.add_trace(go.Scatter(x=-data_30gen_score[:, 1], y=data_30gen_score[:, 0],
                    mode='markers',
                    name='markers'))

# Line; Pareto Frontier ~ 10 gen
# -[:, 1] = -flops => flops,  [:, 0] = val_accuracy
fig.add_trace(go.Scatter(x=-pareto_front_10gen_sorted[:, 1], y=pareto_front_10gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 10 gen'))

# Line; Pareto Frontier ~ 20 gen
fig.add_trace(go.Scatter(x=-pareto_front_20gen_sorted[:, 1], y=pareto_front_20gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 20 gen'))

# Line; Pareto Frontier ~ 30 gen
fig.add_trace(go.Scatter(x=-pareto_front_30gen_sorted[:, 1], y=pareto_front_30gen_sorted[:, 0],
                    mode='lines+markers',
                    name='Pareto Frontier ~ 30 gen'))

# Figure Layout
fig.update_layout(
    title=title,
    xaxis=dict(
        title=x_title,
    ),
    yaxis=dict(
        title=y_title,
    ),
)

fig.show()

##### [3] GA 의 pareto frontier 와 random search 의 pareto frontier 비교

In [127]:
# 1. data_path => 각 gen 마다의 df 구축
data_path_GA = '../logs/__New_main_experiment_1_GA_30gen/'
_, _, data_30gen_GA = make_score_array(data_path_GA)

data_path_RS = '../logs/__New_main_experiment_1_RandomSerach_30gen/'
_, _, data_30gen_RS = make_score_array(data_path)

# 2. flops 에 - 붙여서, 'score'로 변환 (=> 둘 다 최대화하도록)
data_30gen_score_GA = np.copy(data_30gen_GA)
data_30gen_score_RS = np.copy(data_30gen_RS)

data_30gen_score_GA[:, 1] = -data_30gen_score_GA[:, 1]
data_30gen_score_RS[:, 1] = -data_30gen_score_RS[:, 1]


# 3. score df 로 변환 후 pareto frontier 구하기
#  3.1. Pareto Frontier 구하기
pareto_30gen_idx_GA = identify_pareto(data_30gen_score_GA)
pareto_front_30gen_GA = data_30gen_score_GA[pareto_30gen_idx_GA]

pareto_30gen_idx_RS = identify_pareto(data_30gen_score_RS)
pareto_front_30gen_RS = data_30gen_score_RS[pareto_30gen_idx_RS]

#  3.2. Pareto frontier 를 선분으로 긋기 위해, 각각 sorting
pareto_front_30gen_df_GA = pd.DataFrame(pareto_front_30gen_GA)
pareto_front_30gen_df_GA.sort_values(0, inplace=True)
pareto_front_30gen_sorted_GA = pareto_front_30gen_df_GA.values

pareto_front_30gen_df_RS = pd.DataFrame(pareto_front_30gen_RS)
pareto_front_30gen_df_RS.sort_values(0, inplace=True)
pareto_front_30gen_sorted_RS = pareto_front_30gen_df_RS.values


# 4. Plot
title = '[Pareto Frontier] GA vs Random Search (Full Training X)'
x_title = 'Flops'
y_title = 'Validation Accuracy'

fig = go.Figure()


# Line; Pareto Frontier - GA
# -[:, 1] = -flops => flops,  [:, 0] = val_accuracy
fig.add_trace(go.Scatter(x=-pareto_front_30gen_sorted_GA[:, 1], y=pareto_front_30gen_sorted_GA[:, 0],
                    mode='lines+markers',
                    name='Genetic Algoritm'))

# Line; Pareto Frontier - Random Search
fig.add_trace(go.Scatter(x=-pareto_front_30gen_sorted_RS[:, 1], y=pareto_front_30gen_sorted_RS[:, 0],
                    mode='lines+markers',
                    name='Random Search'))

# Figure Layout
fig.update_layout(
    title=title,
    xaxis=dict(
        title=x_title,
    ),
    yaxis=dict(
        title=y_title,
    ),
)

fig.show()

## GA 에서 population 내에서의 val_acc, flops 어떻게 찾아나가는지 확인해보기

- generation 을 누적해서 위, 아래를 min, max로 보는게 더 '정보'를 많이 줄듯

In [156]:
data_path = '../logs/__New_main_experiment_1_GA_30gen/'

with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]
gen_num = []
for i in range(niter):
    gen_num.extend([i for j in range(npop)])
    temp = [train_log[str(i)][j][1] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))

# 0 : acc , 1: Flops
gr_type = 0

objs[:,:,0]= -1*objs[:,:,0]

val_acc = objs[:,:,0].reshape(-1).tolist()  # val_accuracy 는 - 붙어있는채로 사용 => minimize 하는 pareto frontier 찾는 함수 그대로 사용
flops = objs[:,:,1].reshape(-1).tolist()
idxs = [i for i in range(len(y1))]

In [132]:
len(val_acc), len(flops)

(620, 620)

In [164]:
# 0~20, 0~40, 0~60, ..., 0~620 단위로 잘라서 보면서
#  각각에 대해 min, max, mean 값 구하기
val_acc_dict = {"min": [], "max":[], "mean":[]}
flops_dict = {"min": [], "max":[], "mean":[]}

idx_to = 20
for i in range(32):  # 0 ~ 31
    min_acc_i = min(val_acc[:idx_to])
    max_acc_i = max(val_acc[:idx_to])
    mean_acc_i = sum(val_acc[:idx_to])/len(val_acc[:idx_to])
    
    min_flops_i = min(flops[:idx_to])
    max_flops_i = max(flops[:idx_to])
    mean_flops_i = sum(flops[:idx_to])/len(flops[:idx_to])
    
    val_acc_dict['min'].append(min_acc_i)
    val_acc_dict['max'].append(max_acc_i)
    val_acc_dict['mean'].append(mean_acc_i)
    
    flops_dict['min'].append(min_flops_i)
    flops_dict['max'].append(max_flops_i)
    flops_dict['mean'].append(mean_flops_i)
    
    idx_to += 20

In [168]:
# Plot
x = list(range(0, 32))  # 0 ~ 31
x_rev = x[::-1]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=val_acc_dict['max']+val_acc_dict['min'][::-1],
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
#     name='Fair',
))


fig.add_trace(go.Scatter(
    x=x, y=val_acc_dict['mean'],
    line_color='rgb(0,100,80)',
    name='[GA] Validation Accuracy',
))

# Figure Layout
fig.update_layout(
    title='[GA] Validation Accuracy - min ~ mean ~ max (cumulative)',
    xaxis=dict(
        title='Generation',
    ),
    yaxis=dict(
        title='Validation Accuracy',
    ),
)

fig.update_traces(mode='lines')
fig.show()

In [169]:
# Plot
x = list(range(0, 32))  # 0 ~ 31
x_rev = x[::-1]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=flops_dict['max']+flops_dict['min'][::-1],
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
#     name='Fair',
))


fig.add_trace(go.Scatter(
    x=x, y=flops_dict['mean'],
    line_color='rgb(0,100,80)',
    name='[GA] Flops',
))

# Figure Layout
fig.update_layout(
    title='[GA] Flops - min ~ mean ~ max (cumulative)',
    xaxis=dict(
        title='Generation',
    ),
    yaxis=dict(
        title='Flops',
    ),
)

fig.update_traces(mode='lines')
fig.show()

##### min-max 말고 standard deviation 으로 mean-std, mean, mean+std

In [170]:
val_acc_dict = {"lower": [], "upper":[], "mean":[]}
flops_dict = {"lower": [], "upper":[], "mean":[]}

idx_from = 0
idx_to = 20
for i in range(31):  # 0 ~ 31
    std_acc_i = np.std(np.array(val_acc[idx_from:idx_to]))
    mean_acc_i = sum(val_acc[idx_from:idx_to])/len(val_acc[idx_from:idx_to])
    
    lower_acc_i = mean_acc_i - std_acc_i
    upper_acc_i = mean_acc_i + std_acc_i
    
    std_flops_i = np.std(np.array(flops[idx_from:idx_to]))                     
    mean_flops_i = sum(flops[idx_from:idx_to])/len(flops[idx_from:idx_to])
    
    lower_flops_i = mean_flops_i - std_flops_i
    upper_flops_i = mean_flops_i + std_flops_i

    val_acc_dict['lower'].append(lower_acc_i)
    val_acc_dict['upper'].append(upper_acc_i)
    val_acc_dict['mean'].append(mean_acc_i)
    
    flops_dict['lower'].append(lower_flops_i)
    flops_dict['upper'].append(upper_flops_i)
    flops_dict['mean'].append(mean_flops_i)
    
    idx_from += 20
    idx_to += 20

In [173]:
# Plot
x = list(range(0, 31))  # 1 ~ 32
x_rev = x[::-1]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=val_acc_dict['upper']+val_acc_dict['lower'][::-1],
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
#     name='Fair',
))


fig.add_trace(go.Scatter(
    x=x, y=val_acc_dict['mean'],
    line_color='rgb(0,100,80)',
    name='[GA] Flops',
))

# Figure Layout
fig.update_layout(
    title='[GA] Validation Accuracy - mean & standard deviation - each population',
    xaxis=dict(
        title='Generation',
    ),
    yaxis=dict(
        title='Flops',
    ),
)

fig.update_traces(mode='lines')
fig.show()

In [172]:
# Plot
x = list(range(0, 31))  # 1 ~ 32
x_rev = x[::-1]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=flops_dict['upper']+flops_dict['lower'][::-1],
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
#     name='Fair',
))


fig.add_trace(go.Scatter(
    x=x, y=flops_dict['mean'],
    line_color='rgb(0,100,80)',
    name='[GA] Flops',
))

# Figure Layout
fig.update_layout(
    title='[GA] Flops - mean & standard deviation - each population',
    xaxis=dict(
        title='Generation',
    ),
    yaxis=dict(
        title='Flops',
    ),
)

fig.update_traces(mode='lines')
fig.show()

# -----------------------------------------------------------

### [낙일] Pareto Frontier 모델 파일 떨구기

In [297]:
inds = np.where(pareto3==1)

In [289]:
data_path = 'C:/Users/82103/Desktop/datascience/1219_code/basic_model/logs/__New_main_experiment_1_GA_30gen/'
with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]

for i in range(niter):
    temp = [train_log[str(i)][j][0] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))
model_info = objs.reshape(-1,21)

In [298]:
model_info[inds]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]])

In [305]:
data_path='../logs/__New_main_experiment_1_RandomSerach_30gen/'
with open(os.path.join(data_path,"train_logging.log")) as json_file:
    data = json.load(json_file)

train_log = data['train_log']
niter = len(train_log)
npop = len(train_log['0'])
objs =[]

for i in range(niter):
    temp = [train_log[str(i)][j][0] for j in range(npop)]
    objs.append(temp)

objs = np.array(objs)
epoch = list(range(niter))
model_info = objs.reshape(-1,21)
model_info[inds]

array([[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0]])